In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Load your dataset
file_path = 'Merged_GNSS_and_SNR_HKKT.csv'
gnss_data = pd.read_csv(file_path)

# Selecting relevant features for modeling
features = ['Phase Residual (m)', 'STD of Pseudorange L1 (m)', 'STD of Pseudorange L2 (m)',
            'Elevation (¡Æ)', 'Azimuth (¡Æ)', 'SNR(dBHz)', 'L1 MP(m)']

# Drop rows with missing values in selected features
gnss_data_clean = gnss_data.dropna(subset=features + ['Pseudorange Residual (m)'])

# Split the data into features (X) and target (y)
X = gnss_data_clean[features]
y = gnss_data_clean['Pseudorange Residual (m)']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
# Initialize and train a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predicting the pseudorange residual using the trained model
y_pred = model.predict(X_test)

# Calculate RMS for M_original (y_test) and M_m (y_pred)
a1 = np.sqrt(mean_squared_error(y_test, np.zeros_like(y_test)))  # RMS of original data
a2 = np.sqrt(mean_squared_error(y_test, y_pred))  # RMS after model prediction

# Calculate improvement rate
improvement_rate = (a1 - a2) / a1

# Print the results
print(f'RMS of Original Pseudorange Residual (M_original): {a1:.4f} meters')
print(f'RMS after model prediction (M_m): {a2:.4f} meters')
print(f'Improvement Rate: {improvement_rate * 100:.2f}%')

RMS of Original Pseudorange Residual (M_original): 0.9288 meters
RMS after model prediction (M_m): 0.6378 meters
Improvement Rate: 31.34%


In [4]:
from keras.models import Sequential
from keras.layers import Dense

# Define the model
fcnn_model = Sequential()
fcnn_model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
fcnn_model.add(Dense(32, activation='relu'))
fcnn_model.add(Dense(1))

# Compile the model
fcnn_model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
fcnn_model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Predicting
y_pred_fcnn = fcnn_model.predict(X_test)

# Calculate RMS for FCNN
a2_fcnn = np.sqrt(mean_squared_error(y_test, y_pred_fcnn))
improvement_rate_fcnn = (a1 - a2_fcnn) / a1

Epoch 1/50


/opt/anaconda3/envs/sklearn-env/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 201us/step - loss: 17325.0195
Epoch 2/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 200us/step - loss: 2205.5457
Epoch 3/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 201us/step - loss: 1732.5709
Epoch 4/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 203us/step - loss: 1008.2004
Epoch 5/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 200us/step - loss: 718.5107
Epoch 6/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 200us/step - loss: 398.9382
Epoch 7/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 200us/step - loss: 274.4516
Epoch 8/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 211us/step - loss: 381.2469
Epoch 9/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 199us/step - loss: 127.5898
Epoch 10/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 200us/step - loss: 82.5442
Epoch 11/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 200us/step - loss: 43.6967
Epoch 12/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 201us/step - loss: 59.7450
Epoch 13/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 200us/step - loss: 13.0544
Epoch 14/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 200us/step 

In [7]:
print(f'FCNN of Original Pseudorange Residual (M_original): {a1:.4f} meters')
print(f'FCNN after model prediction (M_m): {a2_fcnn:.4f} meters')
print(f'Improvement Rate: {improvement_rate_fcnn * 100:.2f}%')

FCNN of Original Pseudorange Residual (M_original): 0.9288 meters
FCNN after model prediction (M_m): 0.9278 meters
Improvement Rate: 0.12%


In [5]:
from sklearn.tree import DecisionTreeRegressor

# Initialize and train the model
dt_model = DecisionTreeRegressor()
dt_model.fit(X_train, y_train)

# Predicting
y_pred_dt = dt_model.predict(X_test)

# Calculate RMS for Decision Tree
a2_dt = np.sqrt(mean_squared_error(y_test, y_pred_dt))
improvement_rate_dt = (a1 - a2_dt) / a1


In [8]:
print(f'DT of Original Pseudorange Residual (M_original): {a1:.4f} meters')
print(f'DT after model prediction (M_m): {a2_dt:.4f} meters')
print(f'Improvement Rate: {improvement_rate_dt * 100:.2f}%')

DT of Original Pseudorange Residual (M_original): 0.9288 meters
DT after model prediction (M_m): 0.6752 meters
Improvement Rate: 27.31%


In [4]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Predicting
y_pred_rf = rf_model.predict(X_test)

# Calculate RMS for Random Forest
a2_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
improvement_rate_rf = (a1 - a2_rf) / a1


In [5]:
print(f'RF of Original Pseudorange Residual (M_original): {a1:.4f} meters')
print(f'RF after model prediction (M_m): {a2_rf:.4f} meters')
print(f'Improvement Rate: {improvement_rate_rf * 100:.2f}%')

RF of Original Pseudorange Residual (M_original): 0.9288 meters
RF after model prediction (M_m): 0.4837 meters
Improvement Rate: 47.92%


In [7]:
from keras.models import Sequential
from keras.layers import Conv1D, Flatten, Dense

# Reshape the input data for CNN (assuming time-series type data)
X_train_cnn = np.expand_dims(X_train, axis=2)
X_test_cnn = np.expand_dims(X_test, axis=2)

# Define the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
cnn_model.add(Flatten())
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dense(1))

# Compile and train the CNN model
cnn_model.compile(optimizer='adam', loss='mean_squared_error')
cnn_model.fit(X_train_cnn, y_train, epochs=50, batch_size=32)

# Predicting
y_pred_cnn = cnn_model.predict(X_test_cnn)

# Calculate RMS for CNN
a2_cnn = np.sqrt(mean_squared_error(y_test, y_pred_cnn))
improvement_rate_cnn = (a1 - a2_cnn) / a1

Epoch 1/50


/opt/anaconda3/envs/sklearn-env/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9387/9387 ━━━━━━━━━━━━━━━━━━━━ 3s 257us/step - loss: 185421.0156
Epoch 2/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 258us/step - loss: 3642.3652
Epoch 3/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 3s 271us/step - loss: 551.3106
Epoch 4/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 256us/step - loss: 241.6526
Epoch 5/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 255us/step - loss: 136.9399
Epoch 6/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 255us/step - loss: 25.3980
Epoch 7/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 255us/step - loss: 8.4746
Epoch 8/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 260us/step - loss: 2.5278
Epoch 9/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 259us/step - loss: 1.1715
Epoch 10/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 257us/step - loss: 0.6436
Epoch 11/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 3s 274us/step - loss: 0.8705
Epoch 12/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 259us/step - loss: 0.8675
Epoch 13/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 263us/step - loss: 0.8611
Epoch 14/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 2s 257us/step - loss: 0.87

In [8]:
print(f'CNN of Original Pseudorange Residual (M_original): {a1:.4f} meters')
print(f'CNN after model prediction (M_m): {a2_cnn:.4f} meters')
print(f'Improvement Rate: {improvement_rate_cnn * 100:.2f}%')

CNN of Original Pseudorange Residual (M_original): 0.9288 meters
CNN after model prediction (M_m): 0.9277 meters
Improvement Rate: 0.12%


In [9]:
from keras.layers import LSTM

# Reshape the input data for RNN (time-series data)
X_train_rnn = np.expand_dims(X_train, axis=2)
X_test_rnn = np.expand_dims(X_test, axis=2)

# Define the RNN (LSTM) model
rnn_model = Sequential()
rnn_model.add(LSTM(50, return_sequences=True, input_shape=(X_train_rnn.shape[1], 1)))
rnn_model.add(LSTM(50))
rnn_model.add(Dense(1))

# Compile and train the RNN model
rnn_model.compile(optimizer='adam', loss='mean_squared_error')
rnn_model.fit(X_train_rnn, y_train, epochs=50, batch_size=32)

# Predicting
y_pred_rnn = rnn_model.predict(X_test_rnn)

# Calculate RMS for RNN
a2_rnn = np.sqrt(mean_squared_error(y_test, y_pred_rnn))
improvement_rate_rnn = (a1 - a2_rnn) / a1

Epoch 1/50


/opt/anaconda3/envs/sklearn-env/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


9387/9387 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - loss: 0.4075
Epoch 2/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.3342
Epoch 3/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.3285
Epoch 4/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.3285
Epoch 5/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.3210
Epoch 6/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.3193
Epoch 7/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.3156
Epoch 8/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.3125
Epoch 9/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.3081
Epoch 10/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.3049
Epoch 11/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 0.3025
Epoch 12/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.2991
Epoch 13/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.2946
Epoch 14/50
9387/9387 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 0.2935
Epoch 15/50
9387/9387 ━━━━

In [10]:
print(f'RNN of Original Pseudorange Residual (M_original): {a1:.4f} meters')
print(f'RNN after model prediction (M_m): {a2_rnn:.4f} meters')
print(f'Improvement Rate: {improvement_rate_rnn * 100:.2f}%')

RNN of Original Pseudorange Residual (M_original): 0.9288 meters
RNN after model prediction (M_m): 0.4903 meters
Improvement Rate: 47.21%


In [13]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.utils import resample
import numpy as np

# Downsample the data (e.g., to 50% of the original data)
X_train_downsampled, y_train_downsampled = resample(X_train, y_train, n_samples=int(len(X_train) * 0.5), random_state=42)

# Scaling the downsampled data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_downsampled)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the SVR model with 'rbf' kernel
svm_model_rbf = SVR(kernel='rbf', cache_size=200, tol=1e-3)  # You can also adjust tol and cache_size for performance
svm_model_rbf.fit(X_train_scaled, y_train_downsampled)

# Predicting
y_pred_svm_rbf = svm_model_rbf.predict(X_test_scaled)

# Calculate RMS for SVR with RBF kernel
a2_svm_rbf = np.sqrt(mean_squared_error(y_test, y_pred_svm_rbf))
improvement_rate_svm_rbf = (a1 - a2_svm_rbf) / a1

print(f'RMS of RBF SVR Model (Downsampled): {a2_svm_rbf:.4f} meters')
print(f'Improvement Rate: {improvement_rate_svm_rbf * 100:.2f}%')


RMS of RBF SVR Model (Downsampled): 0.5641 meters
Improvement Rate: 39.26%
